In this notebook, we build on the model training of Exploration 2 in the real-time listening and processing of Exploration 3, to build a single unified workflow. Running this allows you to, in this one self-contained notebook,
* specify the noises you want your model to recognize
* record training and testing data for those noises
* train a model on the data, and evaluate its performance
* use the model with a listener to recognize noises in real-time and act on them